In [34]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [35]:
url = 'http://csbl.bmb.uga.edu/DOOR/search.php?keyword=Pseudomonas+aeruginosa'
driver = webdriver.Firefox(executable_path='/Users/gabrielducrocq/Downloads/geckodriver')
driver.get(url)
select = Select(driver.find_element_by_name('DataTables_Table_0_length')) 
select.select_by_value('-1')
time.sleep(60)
html_source = driver.page_source 

In [36]:
driver.close()

In [37]:
soup = bs(html_source)

/Users/gabrielducrocq/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/gabrielducrocq/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [39]:
entries = soup.find_all("table")[0].find_all("tbody")[1].find_all("tr")

In [41]:
cels = [entrie.find_all("td") for entrie in entries]

In [43]:
cels[0]

[<td class=""><a href="operon.php?id=12029">12029</a></td>,
 <td class=""><span class="highlight">Pseudomonas</span> <span class="highlight">aeruginosa</span> PAO1 : NC_002516</td>,
 <td class="">4</td>,
 <td class="">PA0001; PA0002; PA0003; PA0004</td>,
 <td class="">dnaA; dnaN; recF; gyrB</td>,
 <td class="">chromosomal replication initiation protein; DNA polymerase III subunit beta; recombination protein F; DNA gyrase subunit B</td>]

In [76]:
ids = [cel[0].find("a").get_text().strip() for cel in cels]
num_operons = [int(cel[2].get_text()) for cel in cels]
genes = [cel[3].get_text().strip() for cel in cels]

In [84]:
df = pd.DataFrame({"Id":ids, "Nb_Operons":num_operons, "Genes":genes})

In [85]:
df["Genes"] = df.Genes.apply(lambda x: [a.strip() for a in x.split(";")])

In [87]:
df.to_csv("Operons.csv")

In [86]:
df["Genes"]

0                         [PA0001, PA0002, PA0003, PA0004]
1                                         [PA0005, PA0006]
2                                         [PA0008, PA0009]
3                                 [PA0016, PA0017, PA0018]
4                                         [PA0021, PA0022]
5                                         [PA0024, PA0025]
6                                 [PA0026, PA0027, PA0028]
7                                 [PA0029, PA0030, PA0031]
8                                         [PA0033, PA0034]
9                                         [PA0035, PA0036]
10                                        [PA0040, PA0041]
11                                        [PA0045, PA0046]
12                                [PA0054, PA0055, PA0056]
13                                        [PA0061, PA0062]
14                                [PA0064, PA0065, PA0066]
15                                        [PA0067, PA0068]
16       [PA0070, PA0071, PA0072, PA0073, PA0074, PA007.

# Learning Com

In [91]:
import numpy as np

In [128]:
class Agent:
    
    def __init__(self, pos, v, gaze, col, mem_bank = [0, 0], goals = [0, 0, 1]):
        self.golas = goals
        
        self.x_pos = pos[0]
        self.y_pos = pos[0]
        
        self.x_velocity = v[0]
        self.y_velocity = v[1]
        
        self.x_gaze = gaze[0]
        self.y_gaze = gaze[1]
        
        self.memory = mem_bank
        self.goals = goals
        self.col = col
        
    def get_position(self):
        return (self.x_pos, self.y_pos)
    
    def get_velocity(self):
        return (self.x_velocity, self.y_velocity)
    
    def get_gaze(self):
        return (self.x_gaze, self.y_gaze)
    
    def get_memory(self):
        return self.memory

    def get_goals(self):
        return self.goals
    
    def get_color(self):
        return self.col
    
    def get_phys_state(self):
        return (self.get_position(), self.get_velocity(), self.get_gaze(), self.get_col)
    
    def set_position(self, pos):
        self.x_pos = pos[0]
        self.y_pos = pos[1]
      
    def set_velocity(self, velocity):
        self.x_velocity = velocity[0]
        self.y_velocity = velocity[1]
        
    def set_gaze(self, gaze):
        self.x_gaze = gaze[0]
        self.y_gaze = gaze[1]
            
    def set_phys_state(self, pos, velocity, gaze):
        self.set_position(pos)
        self.set_velocity(velocity)
        self.set_gaze(gaze)

In [131]:
class Environment:
    
    def __init__(self, nb_agent, borders = 1000, delta_t = 0.1, damping_coef = 0.5):
        self.nb_agent= nb_agent
        self.borders = borders
        self.damping_coef = damping_coef
        self.delta_t = delta_t
        self.list_agents = []
        for i in range(nb_agent):
            pos = tuple(np.random.randint(0, self.borders, 2))
            v = (0, 0)
            gaze = tuple(np.random.randint(0, self.borders, 2))
            col = np.random.randint(0, 255, 3)
            ag = Agent(pos, v, gaze, col)
            self.list_agents.append(ag)
            
    def get_agents(self):
        return self.list_agents
            
        
    def compute_new_pos(self, pos, velocity):
        return pos + velocity*self.delta_t
    
    def compute_new_velocity(self, pos, u, ag_indexes):
        return self.damping_coef*pos + u #COMPUTE THE FORCES !!
    
    def compute_new_phys_state(self, up, uv):
        for i in range(self.nb_agents):
            ag = self.list_agents[i]
            old_pos = ag.get_position
            old_velocity = ag.get_velocity
            new_pos = self.compute_new_pos(old_pos, old_velocity)
            new_velocity = self.compute_new_velocity(old_velocity, up[i], [k for k in range(self.nb_agent)])
            
            list_phys_states.append((new_pos, new_velocity, uv))
        
        return list_phys_states
    
    def set_new_phys_states(self, list_phys_states):
        for i in range(self.nb_agents):
            ag = self.list_agent[i]
            pos, vel, gaze =list_phys_states[i]
            ag.set_phys_state(pos, vel, gaze)
            
        
    def take_step(self, up, uv):
        list_phys_states = self.compute_new_phys_states(up, uv)
        self.set_new_phys_states(list_phys_states)
        
            
            
            
        

In [132]:
env = Environment(4)

In [125]:
l

[1, 2, 3]

In [126]:
y

[1, 3]

In [127]:
range(3).remove(2)

AttributeError: 'range' object has no attribute 'remove'